In [1]:
import pandas as pd
import re
from ast import literal_eval
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '90'
formatted_attribute = 'Core Diameter'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
curation_col = f'Q:{attribute}'
field1='Width [Nom]'
fields = [field1]

In [3]:
df_cur, df_custom=parameters(customer_id, formatted_attribute, field1)
df_custom=custom_field(df_custom,field1,formatted_attribute)

1282


In [4]:
# matchcust=df_custom[['external_id',curation_col]]
# print(len(matchcust))
# rounding(df_custom,curation_col)

In [5]:
del df_cur['bucket_id']
del df_cur['product_id']
del df_cur['image_url']
del df_cur['value']
del df_cur['custom_fields']

# General Regex

In [6]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3Le?n?g?t?h?\b)|()'''
df_cur['matches_length'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!up.to.)(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}He?i?g?h?t?(?! Adjustable)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'''        
df_cur['matches_height'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Wi?d?t?h?\b)|()'''
df_cur['matches_width'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'''(?i)((?<!\d)(?<!\w )(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)[\W]{0,3}Diam?e?t?e?r?\b)|(.{0,20}core.?dia.{0,20})|(\d.{0,5}Core)|()'''
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
# regex_pattern_blank=r'\d'                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?x?\W?(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)?\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)?\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|((?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)\W{0,3}(?:"|'|''|inc?h?|foot|fe?e?t|”)\s?(?:le?n?g?t?h?\b|(?:Wdith|W\b)|he?i?g?h?t?\b|de?p?t?h?\b|diam?|Round)?\s?)|()'''
df_cur['blank_product'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [7]:
print(len(df_cur[df_cur['matches_diameter'].astype(str)!='[]']))
print(len(df_cur[df_cur['matches_diameter'].astype(str)=='[]']))
df_cur['matches_diameter'].explode().value_counts()

299
983


3" Core       107
1" Core        66
3" core        60
1" core        51
3� core        12
1.5" Core       1
0", 1"Core      1
1.5� Core       1
Name: matches_diameter, dtype: int64

# Core Diameter

In [8]:
from natsort import natsorted
regex_pattern=r'''(?i)(\d(?:\\?\"|\”|inc?h?|''|�)\W{0,3}Core)|()'''           
df_cur['matches'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# Filter out the SKUs that dont have numbers
pat_number=r'\d'
df_cur['new_matches'] = df_cur['matches'].apply(lambda x: re_extract(pat_number, str(x)))
lengthNew=df_cur[(df_cur['new_matches'].astype(str)!='[]')&(df_cur['matches'].astype(str)!='[]')]
del lengthNew['new_matches']

lengthNew['matches']=lengthNew['matches'].apply(lambda x: natsorted(x)).apply(lambda x: sorted(x))
lengthNew['matchez']=lengthNew['matches'].apply(lambda x: re.sub(r'''(?i)((?:\\?\"|\”|inc?h?|''|�)\W{0,2}Core)''',' in',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))


curate(lengthNew, 'matchez', formatted_attribute, 'a-eghj-mo-su-z')
print(len(lengthNew))
matchesnew=lengthNew[['external_id',curation_col]]
matchesnew[curation_col].explode().value_counts()
rounding(lengthNew, 'matchez','a-eghj-mo-su-z')

299


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, matches_length, matches_height, matches_width, matches_diameter, blank_product, blank_long, matches, matchez, Q:core_diameter, rounding]
Index: []

In [9]:
matchesnew[curation_col].explode().value_counts()

["3 in"]    179
["1 in"]    118
["5 in"]      2
Name: Q:core_diameter, dtype: int64

# N/A Values

In [10]:
x=df_cur[df_cur['matches'].astype(str)=='[]']
print(len(x[x['blank_long'].astype(str)!='[]']))
x[x['blank_long'].astype(str)!='[]'][0:500]

959


buckets external_id  \
15    Tapes  5637177589   
16    Tapes  5637177590   
17    Tapes  5637187083   
21    Tapes  5637208628   
23    Tapes  5637209330   
24    Tapes  5637209333   
25    Tapes  5637218945   
26    Tapes  5637227844   
27    Tapes  5637211883   
30    Tapes  5637211910   
41    Tapes  5637219333   
42    Tapes  5637220535   
43    Tapes  5637232480   
49    Tapes  5637232857   
50    Tapes  5637232897   
51    Tapes  5637232900   
61    Tapes  5637259868   
62    Tapes  5637259870   
64    Tapes  5637259567   
65    Tapes  5637279636   
69    Tapes  5637283655   
73    Tapes  5637284376   
74    Tapes  5637283873   
76    Tapes  5637283910   
77    Tapes  5637283930   
78    Tapes  5637284244   
79    Tapes  5637284257   
81    Tapes  5637284285   
82    Tapes  5637294173   
84    Tapes  5637295114   
85    Tapes  5637296057   
86    Tapes  5637296084   
87    Tapes  5637296107   
88    Tapes  5637296174   
89    Tapes  5637296175   
90    Tapes  5637298287   
91    Tapes  5637293851   
92    Tapes  5637296856   
93    Tapes  5637296586   
94    Tapes  5637296631   
95    Tapes  5637296632   
96    Tapes  5637296648   
97    Tapes  5637296691   
98    Tapes  5637296757   
99    Tapes  5637296781   
100   Tapes  5637297668   
101   Tapes  5637296456   
102   Tapes  5637296482   
103   Tapes  5637297482   
107   Tapes  5637308180   
108   Tapes  5637315620   
109   Tapes  5637323102   
110   Tapes  5637323103   
112   Tapes  5637313022   
113   Tapes  5637314705   
120   Tapes  5637316136   
123   Tapes  5637329913   
124   Tapes  5637329914   
125   Tapes  5637329915   
126   Tapes  5637329916   
128   Tapes  5637328267   
129   Tapes  5637328268   
130   Tapes  5637336290   
131   Tapes  5637337136   
132   Tapes  5637337137   
133   Tapes  5637337138   
134   Tapes  5637337139   
135   Tapes  5637336291   
136   Tapes  5637336292   
137   Tapes  5637336293   
140   Tapes  5637346542   
144   Tapes  5637355626   
145   Tapes  5637355627   
146   Tapes  5637355630   
148   Tapes  5637353847   
149   Tapes  5637353845   
150   Tapes  5637353846   
151   Tapes  5637353848   
158   Tapes  5637353651   
159   Tapes  5637353602   
162   Tapes  5637358714   
163   Tapes  5637367183   
164   Tapes  5637367184   
165   Tapes  5637367185   
166   Tapes  5637367182   
183   Tapes  5637403456   
185   Tapes  5637402552   
189   Tapes  5637408998   
190   Tapes  5637409000   
194   Tapes  5637422623   
195   Tapes  5637422624   
196   Tapes  5637422625   
197   Tapes  5637424071   
198   Tapes  5637432197   
199   Tapes  5637432198   
200   Tapes  5637432199   
201   Tapes  5637432200   
202   Tapes  5637432201   
203   Tapes  5637432202   
204   Tapes  5637432203   
205   Tapes  5637432204   
206   Tapes  5637432205   
207   Tapes  5637432206   
208   Tapes  5637432207   
209   Tapes  5637432208   
210   Tapes  5637432209   
211   Tapes  5637432210   
212   Tapes  5637432211   
213   Tapes  5637432212   
214   Tapes  5637432213   
215   Tapes  5637432214   
216   Tapes  5637432215   
217   Tapes  5637432216   
218   Tapes  5637432217   
219   Tapes  5637432218   
220   Tapes  5637432219   
221   Tapes  5637432220   
222   Tapes  5637432221   
223   Tapes  5637432222   
224   Tapes  5637432223   
225   Tapes  5637432224   
226   Tapes  5637432225   
227   Tapes  5637432226   
228   Tapes  5637432227   
229   Tapes  5637432228   
230   Tapes  5637432545   
231   Tapes  5637433481   
232   Tapes  5637432606   
233   Tapes  5637432607   
234   Tapes  5637432512   
235   Tapes  5637432513   
236   Tapes  5637432514   
237   Tapes  5637432515   
238   Tapes  5637432516   
239   Tapes  5637432517   
240   Tapes  5637432518   
241   Tapes  5637432519   
242   Tapes  5637432520   
243   Tapes  5637432521   
244   Tapes  5637432522   
245   Tapes  5637432523   
246   Tapes  5637432524   
247   Tapes  5637432525   
248   Tapes  5637432526   
249   Tapes  5637432527   
250   Tapes  5637432528   
251   Tapes  5637432529   
252   Ta

In [11]:
df_na=df_cur[df_cur['matches_diameter'].astype(str)=='[]']
# regex_pattern_na=r'''(?i)((?<!Cord Measures )(?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)(?:\d+(?:\.\d+|\D?\d+\/\d+)?|\d+\/\d+)['"”].{0,2}(?:[LWHD]|diam)?)|(measure)|()'''
# df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

# df_na=df_na[(df_na['na_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
# matchesna

983


In [12]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcust))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

matchesnew
matchesna


In [ ]:
stop

# send to the folder for upload

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesnew)

In [15]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name

# def looks_good(customer, attribute, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

# looks_good('WBMason', attribute, matchcust)

In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('WBMason', attribute, matchesna)